In [56]:
import pandas as pd
import numpy as np
from datetime import datetime
from scipy.optimize import minimize

In [57]:
df = pd.read_csv('/Users/dominicprenovost/Programmation/TP2-PF-management/48_Industry_Portfolios.CSV', header=6)
df = df.rename(columns={'Unnamed: 0': 'Date'})

df_48ind = df.iloc[:1171].copy()
df_48ind['Date'] = pd.to_datetime(df_48ind['Date'], format='%Y%m')
df_48ind.set_index('Date', inplace=True)
df_48ind = df_48ind.apply(pd.to_numeric, errors='coerce')

df_numfirm = df.iloc[2564-20:3735-20].copy()
df_numfirm['Date'] = pd.to_datetime(df_numfirm['Date'], format='%Y%m')
df_numfirm.set_index('Date', inplace=True)
df_numfirm = df_numfirm.apply(pd.to_numeric, errors='coerce')

df_avgsize = df.iloc[3739-22:4910-22].copy()
df_avgsize['Date'] = pd.to_datetime(df_avgsize['Date'], format='%Y%m')
df_avgsize.set_index('Date', inplace=True)
df_avgsize = df_avgsize.apply(pd.to_numeric, errors='coerce')


## 1) La capitalisation boursière en tant que taille moyenne de l'entreprise x nombre d'entreprises

In [58]:
market_cap = df_numfirm.multiply(df_avgsize, axis=0)
market_cap

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1926-07-01,299.40,1247.60,-0.00,21.36,955.52,13.00,171.84,4.33,178.16,220.32,...,212.10,4977.87,2.38,1539.45,64.92,43.50,95.46,45.84,49.60,97.00
1926-08-01,306.18,1246.00,-0.00,20.25,967.52,14.12,176.04,6.50,177.04,237.96,...,227.16,5058.17,1.80,1536.81,66.00,45.51,94.95,47.12,46.65,102.04
1926-09-01,313.02,1278.00,-0.00,25.74,1024.48,16.50,174.60,9.29,169.44,231.48,...,220.92,5281.55,1.90,1521.63,65.64,50.91,97.38,48.94,46.80,108.84
1926-10-01,308.73,1288.40,-0.00,26.76,1030.72,17.88,183.48,8.83,169.84,228.36,...,208.62,5269.14,1.76,1522.95,64.80,49.38,97.98,47.44,44.45,104.64
1926-11-01,307.02,1236.00,-0.00,25.86,1041.28,17.62,174.06,9.31,161.12,228.36,...,196.80,5115.11,1.48,1485.99,61.98,43.56,93.78,44.66,43.95,95.76
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,41006.68,505297.52,346342.32,346278.68,229020.70,41627.04,394473.12,27397.08,677271.34,252346.56,...,75465.50,812140.55,519543.20,3359902.26,696424.79,2626741.91,1581703.20,68858.24,1204022.82,1219430.46
2023-10-01,41351.20,482695.72,322070.22,328411.27,219676.30,38554.32,350182.43,25643.28,630874.36,240484.16,...,73352.80,769734.55,505316.24,3152332.77,665101.66,2520866.93,1618384.68,61585.28,1159465.58,1184147.60
2023-11-01,38939.89,453495.00,320819.40,314148.67,210891.15,32941.16,359816.49,25072.20,626535.45,250580.48,...,70811.40,721091.80,497719.17,3174484.60,653520.80,2442166.45,1674202.86,55927.10,1082860.33,1162998.24


## 2) Le rapport entre la valeur comptable (BE : book equity) et la valeur de marché (ME : market equity), c'est-à-dire le rapport book-to-market, en utilisant les données "Sum of BE / Sum of ME".

### (Remarque : les données "Somme de BE / Somme de ME" sont annuelles et doivent être converties en données mensuelles en supposant que le ratio reste constant entre juillet de l'année s et juin de l'année s + 1. Par exemple, le ratio de 1926 est le ratio qui doit être utilisé pour la période allant de juillet 1926 à juin 1927).

In [65]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_monthly = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

# Conversion de la colonne 'Date' en format datetime avec un format spécifié
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'], format='%Y')

# Ajustement de la Date au 1er juillet
#df_monthly['Date'] = df_monthly['Date'] + pd.offsets.MonthBegin(2)
df_monthly

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
2,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
3,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
4,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1172,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1173,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1174,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


In [66]:
df_BtoM = df.iloc[4890:4988].copy()
df_BtoM = df_BtoM.apply(pd.to_numeric, errors='coerce')

df_monthly = df_BtoM.loc[df_BtoM.index.repeat(12)].reset_index(drop=True)

# Conversion de la colonne 'Date' en format datetime avec un format spécifié
df_monthly['Date'] = pd.to_datetime(df_monthly['Date'], format='%Y')

df_monthly

,Date,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
0,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
1,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
2,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
3,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
4,1926-01-01,0.65,0.63,-99.99,2.54,0.79,1.99,0.81,4.34,0.38,...,0.97,1.41,2.35,0.44,0.66,0.66,0.62,0.88,0.89,0.63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1171,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1172,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1173,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48
1174,2023-01-01,0.65,0.43,0.10,0.13,0.28,0.39,0.17,0.59,0.14,...,0.42,0.22,0.27,0.16,0.14,0.65,0.37,0.43,0.42,0.48


## 3) Le momentum de chaque industrie est le rendement moyen de cette industrie au cours des 12 derniers mois, y compris le mois t.

In [77]:
df_mom = df_48ind.copy()

# Calculer la moyenne mobile sur 12 mois pour chaque industrie
momentum_df = df_mom.rolling(window=12).mean()

# Afficher le DataFrame du momentum
momentum_df.dropna(inplace=True)

momentum_df


,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
1927-06-01,1.379167,1.680000,-99.990000,4.175833,3.114167,1.063333,0.145833,6.995000,0.960833,1.729167,...,0.235833,1.900000,-6.765000,0.904167,1.733333,-1.163333,2.417500,1.382500,0.719167,0.238333
1927-07-01,1.685833,2.072500,-99.990000,4.990833,3.372500,0.562500,0.015000,2.766667,1.459167,1.556667,...,0.595833,2.238333,-3.856667,1.740833,1.967500,-1.269167,3.576667,1.467500,1.263333,0.372500
1927-08-01,1.665833,2.049167,-99.990000,2.406667,3.150000,-0.223333,0.581667,-0.760000,1.800833,1.895833,...,1.107500,1.695833,-4.520833,2.443333,2.024167,-1.835000,3.371667,1.193333,1.492500,0.302500
1927-09-01,2.324167,2.398333,-99.990000,2.655833,3.430833,-0.885833,0.440000,-0.255833,1.950833,2.325833,...,1.850000,2.048333,-3.398333,3.056667,2.565833,-1.310000,3.991667,1.430000,2.430000,0.660000
1927-10-01,1.879167,2.559167,-99.990000,2.865000,3.315833,-0.875000,0.903333,-0.981667,2.187500,3.120000,...,2.285000,1.882500,-3.931667,2.820833,2.762500,-0.107500,5.244167,1.555000,2.464167,1.065833
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-01,-0.443333,0.130833,0.800833,0.579167,1.386667,0.681667,2.873333,2.539167,1.005833,1.811667,...,1.295000,1.810000,2.147500,1.186667,1.638333,0.859167,0.887500,1.377500,1.430000,2.147500
2023-10-01,-2.074167,-1.222500,0.175000,-0.571667,0.047500,-0.840000,1.854167,1.636667,0.530000,1.255833,...,0.534167,0.715833,0.934167,1.092500,0.615000,-0.496667,0.110000,0.045833,-0.054167,1.101667
2023-11-01,-2.465833,-1.114167,-0.067500,-0.516667,-0.021667,-0.790833,2.170833,0.989167,0.159167,0.682500,...,0.526667,0.895000,1.174167,1.427500,0.755000,0.169167,0.320833,-0.030833,0.355000,1.069167
